<a href="https://colab.research.google.com/github/JonJonesBR/MESTRE_RPG_GEMINI/blob/main/RPG_AUDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q google-generativeai edge-tts playsound nest_asyncio gradio Pillow

import gradio as gr
import google.generativeai as genai
import time
import os
import io
import sys
import asyncio
import re

# Configuração para permitir loops asyncio aninhados
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("nest_asyncio aplicado.")
except ImportError:
    print("AVISO: nest_asyncio não encontrado.")
except RuntimeError as e:
    if "cannot apply nest_asyncio" in str(e) or "loop is already running" in str(e):
        print(f"nest_asyncio: {e}. Provavelmente já está gerenciado pelo ambiente (ex: Gradio).")
    else:
        print(f"AVISO: nest_asyncio: {e}")

_edge_tts_available = False
try:
    import edge_tts
    _edge_tts_available = True
except ImportError:
    print("AVISO: Biblioteca edge-tts não encontrada. A funcionalidade de voz não estará disponível.")

API_KEY_FILENAME = ".gemini_api_key.txt"

DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (objetivo direto)", "id": "curta", "turns": 10},
    "2": {"name": "Média (mais exploração)", "id": "media", "turns": 20},
    "3": {"name": "Longa (desenvolvimento e clímax)", "id": "longa", "turns": 40}
}

RPG_THEMES_CONFIG = {
    "medieval": "Medieval Clássico",
    "fantasia": "Alta Fantasia",
    "futurista": "Futurista (Sci-Fi)",
    "cyberpunk": "Cyberpunk Noir",
    "steampunk": "Steampunk",
    "velho_oeste": "Velho Oeste",
    "pos_apoc": "Pós-Apocalíptico"
}

THEME_AVATARS = {
    "medieval": (None, "🏰"),
    "fantasia": (None, "🧙"),
    "futurista": (None, "🤖"),
    "cyberpunk": (None, "👾"),
    "steampunk": (None, "⚙️"),
    "velho_oeste": (None, "🤠"),
    "pos_apoc": (None, "☣️"),
    "default": (None, "🤖")
}

THEME_CLASSES = {
    "medieval": ["Cavaleiro(a) Honrado(a)", "Mago(a) Erudito(a)", "Arqueiro(a) Furtivo(a)", "Clérigo(a) Devoto(a)", "Bardo(a) Viajante"],
    "fantasia": ["Guerreiro(a) Élfico(a)", "Feiticeira Elemental", "Ladino(a) Halfling", "Paladino(a) Draconato(a)", "Druida Metamorfo"],
    "futurista": ["Piloto(a) de Mecha", "Engenheiro(a) Cibernético(a)", "Agente Secreto(a) Biônico(a)", "Explorador(a) Estelar", "Diplomata Galáctico(a)"],
    "cyberpunk": ["Samurai das Ruas", "Netrunner (Hacker)", "Detetive Bio-modificado(a)", "Traficante de Informações", "Médico(a) Clandestino(a)"],
    "steampunk": ["Inventor(a) Genial", "Caçador(a) de Autômatos", "Aeronauta Destemido(a)", "Detetive a Vapor", "Engenhateiro(a) Real"],
    "velho_oeste": ["Pistoleiro(a) Solitário(a)", "Caçador(a) de Recompensas", "Médico(a) Itinerante", "Jogador(a) de Cartas", "Xerife da Fronteira"],
    "pos_apoc": ["Sobrevivente Nato(a)", "Batedor(a) de Ermos", "Mecânico(a) de Sucata", "Líder de Facção", "Mutante Errante"]
}

model_text = None
MODEL_NAME_TEXT = "gemini-2.0-flash-lite"

generation_config_text = {"temperature": 0.85, "top_p": 0.95, "top_k": 40}
safety_settings_text = [
    {"category": c, "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]
]

async def _speak_text_async_gradio(text, voice_name="pt-BR-ThalitaNeural", rate="+0%", pitch="+0Hz"):
    if not _edge_tts_available or not text or not text.strip(): return None
    text_cleaned = re.sub(r'[\*#_`]', ' ', text)
    text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    if not text_cleaned: return None
    try:
        communicate = edge_tts.Communicate(text_cleaned, voice_name, rate=rate, pitch=pitch)
        audio_bytes_io = io.BytesIO()
        async for chunk in communicate.stream():
            if chunk["type"] == "audio": audio_bytes_io.write(chunk["data"])
        audio_bytes_io.seek(0)
        if audio_bytes_io.getbuffer().nbytes > 0:
            temp_filename = f"temp_audio_{int(time.time()*1000)}_{hash(text_cleaned)}.mp3"
            with open(temp_filename, "wb") as f: f.write(audio_bytes_io.getvalue())
            return temp_filename
        return None
    except Exception as e_tts:
        print(f"[TTS Edge Erro Geral: {e_tts}]")
        return None

async def speak_text_for_gradio(text, voice_name="pt-BR-ThalitaNeural"):
    if not _edge_tts_available or not text or not text.strip(): return None
    return await _speak_text_async_gradio(text, voice_name)

def configure_gemini_api_gradio_direct(api_key_input):
    global model_text
    feedback_list = []
    api_key_val = api_key_input.strip() if api_key_input else ""
    if not api_key_val or not api_key_val.startswith("AIza") or len(api_key_val) <= 20:
        feedback_list.append("Formato de chave API inválido.")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover '{API_KEY_FILENAME}': {e_rem}")
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)
    try:
        with open(API_KEY_FILENAME, "w") as f: f.write(api_key_val)
        feedback_list.append(f"Chave API salva em '{API_KEY_FILENAME}'.")
    except Exception as e:
        feedback_list.append(f"Não foi possível salvar a chave API: {e}.")
    try:
        genai.configure(api_key=api_key_val)
        model_text = genai.GenerativeModel(
            model_name=MODEL_NAME_TEXT,
            generation_config=generation_config_text,
            safety_settings=safety_settings_text
        )
        feedback_list.append(f"API Gemini configurada. Modelo de Texto '{model_text.model_name}' pronto.")
        return "\n".join(feedback_list), gr.update(visible=False), gr.update(visible=True)
    except Exception as e:
        feedback_list.append(f"Erro CRÍTICO ao configurar API Gemini (Modelo: {MODEL_NAME_TEXT}): {e}")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido devido a erro crítico.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover chave: {e_rem}")
        model_text = None
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)

def load_api_key_on_startup_gradio():
    global model_text
    api_key_loaded_val = ""
    feedback = []
    api_section_vis = gr.update(visible=True)
    game_setup_vis = gr.update(visible=False)
    if os.path.exists(API_KEY_FILENAME):
        try:
            with open(API_KEY_FILENAME, "r") as f: api_key_loaded_val = f.read().strip()
            if api_key_loaded_val and api_key_loaded_val.startswith("AIza") and len(api_key_loaded_val) > 20:
                feedback.append("Chave API carregada de arquivo local.")
                config_feedback_str, api_section_vis, game_setup_vis = configure_gemini_api_gradio_direct(api_key_loaded_val)
                feedback.append(config_feedback_str)
            else:
                if os.path.exists(API_KEY_FILENAME): os.remove(API_KEY_FILENAME)
                feedback.append(f"Chave em '{API_KEY_FILENAME}' inválida, arquivo removido.")
                api_key_loaded_val = ""
        except Exception as e:
            feedback.append(f"Erro ao ler '{API_KEY_FILENAME}': {e}")
            api_key_loaded_val = ""
    if not model_text:
        if not any("API Gemini configurada" in s for s in feedback) and not any("Erro CRÍTICO" in s for s in feedback):
            feedback.append("Insira sua chave API Gemini.")
        api_section_vis = gr.update(visible=True)
        game_setup_vis = gr.update(visible=False)
    initial_classes = THEME_CLASSES.get("medieval", [])
    initial_class_radio_update = gr.update(choices=initial_classes, value=initial_classes[0] if initial_classes else None)
    return "\n".join(feedback), api_section_vis, game_setup_vis, api_key_loaded_val, initial_class_radio_update

def mestre_responde(prompt_para_o_mestre):
    global model_text
    if not model_text: return "Erro: Modelo de Texto Gemini não inicializado. Verifique a API Key."
    try:
        response = model_text.generate_content(prompt_para_o_mestre)
        return response.text
    except Exception as e:
        return f"Erro na API Gemini (Texto): {e}"

def parse_initial_setup_from_gemini(response_text):
    data = {
        "current_objective": "Explorar o desconhecido e descobrir seu propósito.",
        "current_location": "Um local misterioso e evocativo.",
        "player_inventory": ["Um item básico de sobrevivência", "Uma lembrança peculiar"],
        "character_lore": "Um indivíduo com um passado nebuloso, buscando um futuro mais claro.",
        "world_lore": "Um mundo vasto e cheio de segredos, perigos e maravilhas."
    }
    try:
        if not response_text or not response_text.strip():
            print("Resposta do setup inicial vazia, usando defaults.")
            return data
        lines = response_text.strip().split('\n')
        for line in lines:
            if ":" not in line: continue
            prefix, value = line.split(":", 1)
            value = value.strip()
            prefix_upper = prefix.upper().strip()
            if prefix_upper == "LORE_MUNDO": data["world_lore"] = value
            elif prefix_upper == "LORE_PERSONAGEM": data["character_lore"] = value
            elif prefix_upper in ["LOCALIZAÇÃO", "LOCALIZACAO"]: data["current_location"] = value
            elif prefix_upper == "OBJETIVO": data["current_objective"] = value
            elif prefix_upper in ["INVENTÁRIO", "INVENTARIO"]:
                items_str = value.replace('[','').replace(']','')
                parsed_items = [item.strip().strip("'\"") for item in items_str.split(',') if item.strip()]
                if parsed_items: data["player_inventory"] = parsed_items
    except Exception as e:
        print(f"Erro no parse_initial_setup: {e}. Resposta recebida:\n---\n{response_text}\n---")
    return data

def _generate_status_string(gs_dict):
    if not gs_dict or not gs_dict.get("player_name"):
        return "Aguardando início da aventura..."
    status_parts = [
        f"**📜 Aventureiro(a):** {gs_dict.get('player_name', 'N/A')} ({gs_dict.get('player_class', 'N/A')})",
        f"**⏳ Turno:** {gs_dict.get('current_turn', 0)} / {gs_dict.get('max_turns', 'N/A')} ({gs_dict.get('chosen_duration_name', 'N/A')})",
        f"**🎯 Objetivo Atual:** {gs_dict.get('current_objective', 'Não definido')}"
    ]
    inventory_list = gs_dict.get('player_inventory', [])
    inventory_str = ", ".join(inventory_list) if inventory_list else "Vazio"
    status_parts.append(f"**🎒 Inventário:** {inventory_str}")
    return "\n\n".join(status_parts)

async def iniciar_aventura_gradio(player_name_input, player_class_choice, custom_class_input, rpg_theme_key, duration_choice_val, gs_dict, ch_list_internal_ai):
    gs = {}
    ch_internal_ai = []
    initial_chatbot_history_ui = []
    log_setup_feedback = []

    gs["player_name"] = player_name_input.strip() or "Aventureiro(a) Anônimo(a)"
    gs["player_class"] = custom_class_input.strip() or player_class_choice
    if not gs["player_class"]:
        default_classes_for_theme = THEME_CLASSES.get(rpg_theme_key, THEME_CLASSES["medieval"])
        gs["player_class"] = default_classes_for_theme[0] if default_classes_for_theme else "Aventureiro(a) Genérico(a)"

    gs["rpg_theme_config_name"] = RPG_THEMES_CONFIG.get(rpg_theme_key, "Medieval Clássico")
    gs["rpg_theme_key"] = rpg_theme_key

    log_setup_feedback.append(f"Nome: {gs['player_name']}, Classe: {gs['player_class']}, Tema: {gs['rpg_theme_config_name']}")

    opt = DURATION_OPTIONS_CONFIG.get(duration_choice_val, DURATION_OPTIONS_CONFIG["1"])
    gs.update({"chosen_duration_id": opt['id'], "chosen_duration_name": opt['name'], "max_turns": opt['turns'], "current_turn": 0})
    log_setup_feedback.append(f"Duração: {gs['chosen_duration_name']} ({gs['max_turns']} turnos)")

    selected_theme_avatars = THEME_AVATARS.get(gs["rpg_theme_key"], THEME_AVATARS["default"])
    chatbot_ui_update_val = None

    prompt_setup = f"""Você é um mestre de RPG criando a base para uma nova aventura.
Tema Escolhido: {gs['rpg_theme_config_name']}
Nome do Jogador: {gs['player_name']}
Classe do Jogador: {gs['player_class']}
Duração da Aventura (tipo): {gs['chosen_duration_name']}
Gere os seguintes elementos para o início da aventura. Seja criativo, conciso e adequado ao tema e à classe do jogador.
Responda APENAS com os seguintes prefixos, cada um em uma nova linha, seguido pelo conteúdo:
LORE_MUNDO: [Uma breve e envolvente descrição do mundo e sua atmosfera principal, alinhada com o tema.]
LORE_PERSONAGEM: [Uma breve história de origem ou motivação para o personagem do jogador ({gs['player_class']}), conectada ao mundo e ao tema.]
LOCALIZAÇÃO: [O nome e uma breve descrição do local inicial onde a aventura começa. Este é o local para a PARTE 2 da narração inicial.]
OBJETIVO: [O primeiro objetivo claro e conciso para o jogador.]
INVENTÁRIO: [Liste de 2 a 4 itens iniciais que o personagem ({gs['player_class']}) possui, temáticos. Ex: item1, item2, item3]"""
    resposta_setup = mestre_responde(prompt_setup)
    resposta_setup_content = str(resposta_setup) if resposta_setup is not None else "O mestre não conseguiu gerar o setup."

    if "Erro: Modelo de Texto Gemini não inicializado" in resposta_setup_content or "Erro na API Gemini" in resposta_setup_content :
        log_setup_feedback.append(f"Falha crítica: {resposta_setup_content}")
        initial_chatbot_history_ui = [{"role": "assistant", "content": resposta_setup_content}]
        chatbot_ui_update_val = gr.update(value=initial_chatbot_history_ui, avatar_images=selected_theme_avatars)
        initial_status_string = _generate_status_string(gs)
        return gs, ch_internal_ai, chatbot_ui_update_val, "\n".join(log_setup_feedback), None, gr.update(visible=True), gr.update(visible=False), initial_status_string

    parsed_data = parse_initial_setup_from_gemini(resposta_setup_content)
    gs.update(parsed_data)
    gs["current_turn"] = 1
    log_setup_feedback.append(f"Lore Mundo: {gs.get('world_lore', 'N/A')}, Lore Personagem: {gs.get('character_lore', 'N/A')}, Objetivo: {gs.get('current_objective', 'N/A')}")

    prompt_contexto = f"""Você é o Mestre de um RPG Textual, preparando a introdução para o jogador {gs['player_name']}.
Esta é a PRIMEIRA narração da aventura (Turno {gs['current_turn']}), então ela deve ser MAIS LONGA e DETALHADA para estabelecer o cenário e o personagem.
Informações Base Geradas:
- Tema: {gs['rpg_theme_config_name']}
- Jogador: {gs['player_name']} (Classe: {gs['player_class']})
- Lore do Mundo: {gs.get('world_lore', 'Um mundo vasto e misterioso.')}
- Lore do Personagem ({gs['player_name']}): {gs.get('character_lore', 'Um aventureiro com um passado intrigante.')}
- Objetivo Inicial: {gs['current_objective']}
- Localização Inicial: {gs['current_location']}
- Inventário Inicial: {', '.join(gs['player_inventory'])}
Instruções: Combine tudo em uma narração fluida (3 partes: Mundo/Personagem, Cena, Chamada para Ação).
Termine convidando o jogador à sua primeira ação. NÃO use os prefixos LORE_MUNDO, etc., na sua narração final."""
    desc_inicial = mestre_responde(prompt_contexto)
    desc_inicial_content = str(desc_inicial) if desc_inicial is not None else "O mestre não conseguiu iniciar a narração..."
    audio_path = None
    initial_status_string = _generate_status_string(gs)

    if "Erro:" in desc_inicial_content or not desc_inicial_content.strip() or desc_inicial_content == "O mestre não conseguiu iniciar a narração...":
        final_desc = f"O Mestre (IA) falhou: {desc_inicial_content or 'Resposta vazia.'}"
        initial_chatbot_history_ui = [{"role": "assistant", "content": final_desc}]
        audio_path = await speak_text_for_gradio(final_desc)
        gs["player_name"] = None
        chatbot_ui_update_val = gr.update(value=initial_chatbot_history_ui, avatar_images=selected_theme_avatars)
        return gs, ch_internal_ai, chatbot_ui_update_val, "\n".join(log_setup_feedback), audio_path, gr.update(visible=True), gr.update(visible=False), initial_status_string

    initial_chatbot_history_ui = [{"role": "assistant", "content": desc_inicial_content}]
    audio_path = await speak_text_for_gradio(desc_inicial_content)
    gs["current_situation"] = desc_inicial_content
    ch_internal_ai.append({'role': 'system', 'parts': [f"Contexto: Tema: {gs['rpg_theme_config_name']}, Mundo: {gs.get('world_lore')}, Personagem: {gs.get('character_lore')} ({gs['player_class']}), Objetivo: {gs['current_objective']}."]})
    ch_internal_ai.append({'role': 'model', 'parts': [desc_inicial_content]})

    chatbot_ui_update_val = gr.update(value=initial_chatbot_history_ui, avatar_images=selected_theme_avatars)
    return gs, ch_internal_ai, chatbot_ui_update_val, "\n".join(log_setup_feedback), audio_path, gr.update(visible=False), gr.update(visible=True), initial_status_string

async def rodada_do_jogo_gradio(acao_jogador_input, gs_dict, ch_list_internal_ai, current_chatbot_hist_ui_dicts, game_ended_flag_val):
    gs = gs_dict.copy()
    ch_internal_ai = list(ch_list_internal_ai)
    updated_chatbot_hist_ui = list(current_chatbot_hist_ui_dicts) if current_chatbot_hist_ui_dicts is not None else []
    current_status_string = _generate_status_string(gs)

    if not acao_jogador_input.strip():
        yield gs, ch_internal_ai, updated_chatbot_hist_ui, None, gr.update(interactive=True, value=acao_jogador_input), game_ended_flag_val, current_status_string
        return

    updated_chatbot_hist_ui.append({"role": "user", "content": str(acao_jogador_input)})
    pending_message_content = "Mestre (Gemini) está ponderando sobre sua ação..."
    updated_chatbot_hist_ui.append({"role": "assistant", "content": pending_message_content})

    yield gs, ch_internal_ai, updated_chatbot_hist_ui, None, gr.update(interactive=False, value=""), game_ended_flag_val, current_status_string

    audio_mestre_path = None
    jogo_terminou_nesta_rodada = game_ended_flag_val

    if acao_jogador_input.lower() == 'sair':
        msg_sair = f"A jornada de {gs.get('player_name', 'O Aventureiro')} chega ao fim por decisão própria. Obrigado por jogar!"
        if updated_chatbot_hist_ui and updated_chatbot_hist_ui[-1]["role"] == "assistant":
            updated_chatbot_hist_ui[-1] = {"role": "assistant", "content": msg_sair}
        else:
            updated_chatbot_hist_ui.append({"role": "assistant", "content": msg_sair})

        audio_mestre_path = await speak_text_for_gradio(msg_sair)
        jogo_terminou_nesta_rodada = True
        gs["current_turn"] = gs.get("max_turns", gs.get("current_turn",0))
        current_status_string = _generate_status_string(gs)
        yield gs, ch_internal_ai, updated_chatbot_hist_ui, audio_mestre_path, gr.update(interactive=False, value=""), jogo_terminou_nesta_rodada, current_status_string
        return

    gs["previous_player_actions"] = (gs.get("previous_player_actions", []) + [str(acao_jogador_input)])[-3:]
    final_turn_narration_override = ""
    general_narration_length_instruction = "REGRA DE EXTENSÃO: Narrações concisas (1-2 parágrafos curtos)."
    progression_hint = ""
    ct, mt, cdn = gs.get('current_turn', 1), gs.get('max_turns', 1), gs.get('chosen_duration_name', '')
    rpg_theme_name = gs.get('rpg_theme_config_name', 'Tema Desconhecido')

    if mt > 0:
        if ct == mt:
            final_turn_narration_override = f"ATENÇÃO ESPECIAL - ÚLTIMO TURNO ({ct}/{mt}): Para esta narração final da aventura '{cdn}', crie uma conclusão impactante, MAIS LONGA e DETALHADA. A regra de concisão NÃO se aplica."
            general_narration_length_instruction = ""
        elif ct > mt * 0.75:
            progression_hint = f"AVISO DE PROGRESSÃO: A aventura '{cdn}' está se aproximando do fim (Turno {ct}/{mt}). Guie para um clímax. Mantenha concisão por enquanto."

    hist_fmt_parts = []
    for entry in ch_internal_ai[-6:]:
        role_label = "Jogador" if entry['role'] == 'user' else ("Contexto" if entry['role'] == 'system' else "Mestre")
        content = entry.get('parts', [''])[0] if entry.get('parts') else ''
        if entry['role'] != 'system': hist_fmt_parts.append(f"{role_label}: {content}")
    hist_fmt = "\n".join(hist_fmt_parts)

    prompt_mestre = f"""Você é o Mestre de um RPG Textual.
{general_narration_length_instruction}
{final_turn_narration_override}
{progression_hint}
Tema: {rpg_theme_name}. Jogador: {gs.get('player_name')} ({gs.get('player_class')}).
Duração: {cdn} ({mt} turnos), Turno Atual: {ct}.
MUNDO (Lore): {gs.get('world_lore')}
PERSONAGEM (Lore): {gs.get('character_lore')}
HISTÓRICO RECENTE:
{hist_fmt or 'Primeira ação após introdução.'}
SITUAÇÃO ATUAL (sua última narração):
{gs.get('current_situation')}
AÇÃO DO JOGADOR: '{str(acao_jogador_input)}'
INVENTÁRIO: {', '.join(gs.get('player_inventory', ['Nada']))}
OBJETIVO ATUAL: {gs.get('current_objective')}
INSTRUÇÕES:
1. Narre a consequência da ação do jogador.
2. Se MUDANÇA SIGNIFICATIVA, adicione AO FINAL da narração (em nova linha):
   NOVO_OBJETIVO: [novo objetivo]
   NOVA_LOCALIZAÇÃO: [nova localização]
   NOVO_INVENTÁRIO: [item1, item2] (lista completa)
3. Termine com uma deixa para a próxima ação (a menos que seja o final).
"""
    resp_mestre_raw = mestre_responde(prompt_mestre)
    resp_mestre_content = str(resp_mestre_raw) if resp_mestre_raw is not None else "O mestre não respondeu."

    final_bot_response_for_ui_content = ""

    if "Erro: Modelo de Texto Gemini não inicializado" in resp_mestre_content or "Erro na API Gemini" in resp_mestre_content :
        final_bot_response_for_ui_content = resp_mestre_content
        audio_mestre_path = await speak_text_for_gradio(final_bot_response_for_ui_content)
        if updated_chatbot_hist_ui and updated_chatbot_hist_ui[-1]["role"] == "assistant":
            updated_chatbot_hist_ui[-1] = {"role": "assistant", "content": final_bot_response_for_ui_content}
        current_status_string = _generate_status_string(gs)
        yield gs, ch_internal_ai, updated_chatbot_hist_ui, audio_mestre_path, gr.update(interactive=True, value=acao_jogador_input), game_ended_flag_val, current_status_string
        return

    narrative_only_for_tts_and_situation = ""
    if "Erro:" in resp_mestre_content or resp_mestre_content == "O mestre não respondeu.":
        final_bot_response_for_ui_content = f"Mestre parece confuso e com problemas técnicos: {resp_mestre_content}"
        narrative_only_for_tts_and_situation = final_bot_response_for_ui_content
        audio_mestre_path = await speak_text_for_gradio(narrative_only_for_tts_and_situation)
        gs["current_turn"] = max(1, gs.get("current_turn", 1) -1)
    else:
        lines = resp_mestre_content.split('\n')
        final_narrative_lines = []
        parsed_updates_for_log = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("NOVO_OBJETIVO:"):
                new_obj = stripped_line.split(":", 1)[1].strip();
                if new_obj: gs["current_objective"] = new_obj; parsed_updates_for_log.append(f"🎯 Objetivo: {new_obj}")
            elif stripped_line.startswith("NOVA_LOCALIZAÇÃO:"):
                new_loc = stripped_line.split(":", 1)[1].strip();
                if new_loc: gs["current_location"] = new_loc; parsed_updates_for_log.append(f"📍 Local: {new_loc}")
            elif stripped_line.startswith("NOVO_INVENTÁRIO:"):
                items_str_full = stripped_line.split(":", 1)[1].strip()
                items_str = items_str_full.replace('[','').replace(']','')
                parsed_items = [item.strip().strip("'\"") for item in items_str.split(',') if item.strip()]
                if items_str_full == "[]" or (not items_str and items_str_full.strip() == "[]"): gs["player_inventory"] = []; parsed_updates_for_log.append(f"🎒 Inventário: Vazio")
                elif parsed_items: gs["player_inventory"] = parsed_items; parsed_updates_for_log.append(f"🎒 Inventário: {', '.join(parsed_items)}")
            else: final_narrative_lines.append(line)

        narrative_only_for_tts_and_situation = "\n".join(final_narrative_lines).strip()

        if not narrative_only_for_tts_and_situation and parsed_updates_for_log:
            final_bot_response_for_ui_content = "O Mestre registra as mudanças no ambiente e no seu personagem."
            if parsed_updates_for_log: final_bot_response_for_ui_content += "\n\n" + "\n".join([f"<i>({update})</i>" for update in parsed_updates_for_log])
        elif not narrative_only_for_tts_and_situation:
            final_bot_response_for_ui_content = "O Mestre contempla em silêncio, talvez aguardando um momento mais oportuno para falar..."
            narrative_only_for_tts_and_situation = "O Mestre contempla em silêncio."
        else:
            final_bot_response_for_ui_content = narrative_only_for_tts_and_situation
            if parsed_updates_for_log: final_bot_response_for_ui_content += "\n\n" + "\n".join([f"<i>({update})</i>" for update in parsed_updates_for_log])

        audio_mestre_path = await speak_text_for_gradio(narrative_only_for_tts_and_situation)
        gs["current_situation"] = narrative_only_for_tts_and_situation
        ch_internal_ai.append({'role': 'user', 'parts': [str(acao_jogador_input)]})
        ch_internal_ai.append({'role': 'model', 'parts': [narrative_only_for_tts_and_situation]})

    if updated_chatbot_hist_ui and updated_chatbot_hist_ui[-1]["role"] == "assistant":
        updated_chatbot_hist_ui[-1] = {"role": "assistant", "content": final_bot_response_for_ui_content}
    else:
        updated_chatbot_hist_ui.append({"role": "assistant", "content": final_bot_response_for_ui_content})


    if gs.get("current_turn", 0) >= gs.get("max_turns", 1) and not jogo_terminou_nesta_rodada:
        final_msg_turns_suffix = f"\nA aventura '{cdn}' ({gs['current_turn']}/{gs['max_turns']}) chegou ao seu clímax natural. O jogo terminou."
        if not any(keyword in final_bot_response_for_ui_content.lower() for keyword in ["fim", "conclusão", "termina", "acabou", "desfecho"]):
            updated_chatbot_hist_ui.append( {"role": "assistant", "content": final_msg_turns_suffix} )
        jogo_terminou_nesta_rodada = True

    if not jogo_terminou_nesta_rodada:
        gs["current_turn"] += 1

    current_status_string = _generate_status_string(gs)

    yield gs, ch_internal_ai, updated_chatbot_hist_ui, audio_mestre_path, gr.update(interactive=not jogo_terminou_nesta_rodada, value=""), jogo_terminou_nesta_rodada, current_status_string

# --- Estilo CSS RPG ---
css_rpg = """
body, .gradio-container { font-family: 'Georgia', serif; background-color: #261f1e; color: #e0e0e0; }
h1, h2, .gr-markdown strong { color: #e49b50; font-family: 'Cinzel', serif; }
.gr-label { color: #c8a077; font-weight: bold; }
.gr-button { background-color: #6d4c41 !important; color: #f5e6c4 !important; border: 1px solid #f5e6c4 !important; font-family: 'Luminari', 'fantasy', serif; box-shadow: 1px 1px 3px #111; }
.gr-button:hover { background-color: #8b6b5c !important; }
.gr-input, .gr-textbox, .gr-dropdown, .gr-radio, .gr-panel, .block { background-color: #3e2723 !important; color: #f5e6c4 !important; border: 1px solid #5d4037 !important; border-radius: 3px; }
.gr-textbox[label*="Log"] textarea, .gr-textbox[label*="Feedback"] textarea { font-family: 'Courier New', monospace; background-color: #311f1b !important; color: #bcaaa4 !important;}
.gr-chatbot { background-color: #3e2723 !important; border: 1px solid #5d4037 !important; }
.gr-chatbot .message-wrap { margin-bottom: 8px !important; }
.gr-chatbot .message.user { background: #5d4037 !important; color: #fff5e1 !important; align-self: flex-end; border-radius: 10px 0px 10px 10px !important; box-shadow: 1px 1px 2px #1b0000;}
.gr-chatbot .message.bot { background: #4e342e !important; color: #fff5e1 !important; align-self: flex-start; border-radius: 0px 10px 10px 10px !important; box-shadow: 1px 1px 2px #1b0000;}
.gr-chatbot .avatar-container img, .gr-chatbot .avatar-container .avatar {
    border-radius: 50%; border: 2px solid #e49b50;
    width: 32px; height: 32px; text-align: center; line-height:32px; font-size: 22px;
    background-color: #4e342e; color: #f5e6c4;
    object-fit: cover; display: flex; align-items: center; justify-content: center;
}
#game_status_display_rpg { background-color: rgba(40, 20, 10, 0.5); padding: 12px; border-radius: 5px; border: 1px solid #785032; margin-bottom: 10px; }
#game_status_display_rpg p { margin-bottom: 0.5em !important; line-height: 1.5; color: #f5e6c4;}
#game_status_display_rpg strong { color: #e49b50 !important; }
.gr-audio { border: 1px solid #5d4037; border-radius: 5px; padding: 5px; background-color: #3e2723;}
.gr-audio audio { width: 100%; }
.gr-tabitem { background-color: #3e2723 !important; border: 1px solid #5d4037 !important; border-radius: 5px;}
.tabs > .tab-nav > button.selected { background-color: #6d4c41 !important; color: #f5e6c4 !important;}
.tabs > .tab-nav > button { background-color: #4e342e !important; color: #bcaaa4 !important;}
"""

with gr.Blocks(theme=gr.themes.Base(), css=css_rpg, title="RPG com Gemini Aprimorado") as demo:
    game_state_store = gr.State({})
    conversation_history_store_ai = gr.State([])
    game_ended_flag = gr.State(False)

    gr.Markdown("# ⚔️ RPG Textual com IA Gemini & Voz 🐉")
    gr.Markdown("Crie seu personagem, escolha um tema e embarque em aventuras narradas por uma Inteligência Artificial!")

    with gr.Tabs():
        with gr.TabItem("⚙️ Configuração & Jogo 🎮"):
            with gr.Row():
                with gr.Column(scale=1, elem_id="config_column"):
                    with gr.Column(visible=True) as api_key_section:
                        gr.Markdown("## 🔑 1. Chave API Gemini")
                        gr.Markdown("Para jogar, você precisa de uma Chave API do Google AI Studio. [Obtenha sua chave aqui!](https://aistudio.google.com/app/apikey)")
                        api_key_input_gradio = gr.Textbox(label="Sua Chave API do Gemini", placeholder="Cole sua chave AIza...", type="password", lines=1)
                        submit_api_key_button = gr.Button("🔗 Configurar API e Modelo")
                        api_key_feedback = gr.Textbox(label="Feedback da Configuração API", lines=3, interactive=False, max_lines=5)

                    with gr.Column(visible=False, elem_id="game_setup_column") as game_setup_block:
                        gr.Markdown("## 🧙‍♂️ 2. Crie Sua Aventura")
                        player_name_gradio = gr.Textbox(label="Nome do Aventureiro(a)", placeholder="Ex: Elara, Kael, Jax")

                        rpg_theme_options_display = [(name, key) for key, name in RPG_THEMES_CONFIG.items()]
                        rpg_theme_choice_gradio = gr.Dropdown(rpg_theme_options_display, label="Escolha o Tema do RPG", value="medieval")

                        player_class_choice_gradio = gr.Radio(label="Escolha sua Classe Principal")
                        custom_class_gradio = gr.Textbox(label="Ou personalize sua classe (opcional)", placeholder="Ex: Caçador de Relíquias")

                        # Definição de duration_options_display movida para dentro do escopo correto
                        duration_options_display = [(v['name'], k) for k, v in DURATION_OPTIONS_CONFIG.items()]
                        duration_choice_gradio = gr.Radio(duration_options_display, label="Duração da Aventura", value="1")

                        start_adventure_button = gr.Button("🚀 Iniciar Aventura!")
                        game_setup_feedback = gr.Textbox(label="Log de Criação da Aventura (Referência)", lines=6, interactive=False, max_lines=10)
                        audio_output_setup = gr.Audio(label="🎧 Narração Inicial do Mestre", autoplay=True)

                with gr.Column(scale=2, visible=False, elem_id="main_game_column") as main_game_block:
                    gr.Markdown("## 🗺️ 3. Sua Aventura se Desenrola...")
                    game_status_display_md = gr.Markdown(elem_id="game_status_display_rpg")

                    chatbot_component_ui = gr.Chatbot(
                        label="Narrativa da Aventura", height=500,
                        type="messages",
                        avatar_images=(None, THEME_AVATARS["default"][1]),
                        autoscroll=True
                    )
                    player_action_input = gr.Textbox(label="O que você faz a seguir?", placeholder="Digite sua ação ou 'sair' para terminar...", show_label=True, lines=2, interactive=True)
                    submit_action_button = gr.Button("➡️ Enviar Ação")
                    audio_output_game = gr.Audio(label="🎤 Narração do Mestre", autoplay=True)

    def update_class_choices_on_theme_change(theme_key):
        classes = THEME_CLASSES.get(theme_key, [])
        if not classes: classes = THEME_CLASSES.get("medieval", ["Classe Padrão"])
        return gr.update(choices=classes, value=classes[0] if classes else None)

    rpg_theme_choice_gradio.change(
        fn=update_class_choices_on_theme_change,
        inputs=[rpg_theme_choice_gradio],
        outputs=[player_class_choice_gradio]
    )

    demo.load(
        load_api_key_on_startup_gradio,
        inputs=[],
        outputs=[api_key_feedback, api_key_section, game_setup_block, api_key_input_gradio, player_class_choice_gradio])

    submit_api_key_button.click(
        configure_gemini_api_gradio_direct,
        inputs=[api_key_input_gradio],
        outputs=[api_key_feedback, api_key_section, game_setup_block])

    def reset_game_state_for_new_adventure():
        default_avatar = THEME_AVATARS["default"]
        return ({}, [], False,
                gr.update(value=[], avatar_images=default_avatar),
                "", None, None,
                gr.update(value=None), gr.update(value=None),
                "Aguardando início da aventura...")

    start_adventure_button.click(
        reset_game_state_for_new_adventure,
        outputs=[game_state_store, conversation_history_store_ai, game_ended_flag,
                 chatbot_component_ui, player_action_input, audio_output_setup, audio_output_game,
                 game_setup_feedback, audio_output_setup, game_status_display_md]
    ).then(
        iniciar_aventura_gradio,
        inputs=[player_name_gradio, player_class_choice_gradio, custom_class_gradio, rpg_theme_choice_gradio, duration_choice_gradio, game_state_store, conversation_history_store_ai],
        outputs=[game_state_store, conversation_history_store_ai, chatbot_component_ui, game_setup_feedback, audio_output_setup, game_setup_block, main_game_block, game_status_display_md]
    )

    player_action_outputs = [
        game_state_store,
        conversation_history_store_ai,
        chatbot_component_ui,
        audio_output_game,
        player_action_input,
        game_ended_flag,
        game_status_display_md
    ]

    def handle_block_visibility_after_turn(game_has_ended_flag_value):
        global model_text
        api_is_configured = bool(model_text)
        if game_has_ended_flag_value:
            return gr.update(visible=not api_is_configured), gr.update(visible=api_is_configured), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

    game_turn_event_args = {
        "fn": rodada_do_jogo_gradio,
        "inputs": [player_action_input, game_state_store, conversation_history_store_ai, chatbot_component_ui, game_ended_flag],
        "outputs": player_action_outputs,
        "show_progress": "full"
    }

    submit_action_button.click(**game_turn_event_args).then(
        fn=handle_block_visibility_after_turn,
        inputs=[game_ended_flag],
        outputs=[api_key_section, game_setup_block, main_game_block],
        show_progress=False
    )
    player_action_input.submit(**game_turn_event_args).then(
        fn=handle_block_visibility_after_turn,
        inputs=[game_ended_flag],
        outputs=[api_key_section, game_setup_block, main_game_block],
        show_progress=False
    )

demo.queue().launch(debug=True, share=True, inbrowser=True)